In [27]:
%pip install -r /home/HackYeah/HackYeah2024/Backend/TaxAssistantServer/requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\dodeu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [25]:
%pwd

'\\\\wsl.localhost\\Ubuntu\\home\\HackYeah\\HackYeah2024\\RAG-System'

In [15]:
# import importlib
# import db_utils
# importlib.reload(db_utils)

In [ ]:
%pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit langchain-ollama --use-pep517

In [17]:
from langchain_community.embeddings import FastEmbedEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from model_utils import rag_prompt
from config import MODEL_NAME, ES_PORT, ES_INDEX_NAME, ES_DISTANCE_STRATEGY, CONTEXT_CHAR_THRESHOLD, CHUNKER_TYPE
import pickle
from langchain_community.document_loaders import PyPDFDirectoryLoader

Prepare chunks

In [18]:
pdf_directory = "./data/college"
document_chunks = PyPDFDirectoryLoader(pdf_directory, recursive=True).load()

In [ ]:
document_chunks

In [ ]:
len(document_chunks)

In [21]:
# save document_chunks to a file
with open("document_chunks_FastEmbedEmbeddings_college.pkl", "wb") as f:
    pickle.dump(document_chunks, f)

In [22]:
# %pip install langchain-elasticsearch langchain-community langchain_experimental langchain_openai tqdm pypdf streamlit fastembed --use-pep517

Prepare database

In [23]:
from langchain_ollama import OllamaEmbeddings
MODEL_NAME="llama3.2"
embedding = OllamaEmbeddings(
    model=MODEL_NAME,
)
db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

In [ ]:
# clear index 
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

In [ ]:
#print type of document_chunks, db_kwargs
print(type(document_chunks), type(db_kwargs))

In [ ]:
db = add_documents_to_db(db=None, document_chunks=document_chunks, db_kwargs=db_kwargs, bulk_upload=True)

In [ ]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Prompt rag

In [ ]:
model = Ollama(model=MODEL_NAME)

response = rag_prompt(
    context_char_threshold=CONTEXT_CHAR_THRESHOLD,
    query = "Chciałbym wypełnić deklarację PCC-3.",
    model=model,
    db=db,
    chunker_type=CHUNKER_TYPE,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )
